In [1]:
import numpy as np
import os
import random
import scipy
from scipy.interpolate import griddata
from scipy import signal
from numpy.fft import fft
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.initializers import he_normal

from sklearn.model_selection import train_test_split

import pandas as pd
import pickle

import seaborn as sns
sns.set(font='Yu Gothic')
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
channels = ['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4']
channels

['AF3',
 'F7',
 'F3',
 'FC5',
 'T7',
 'P7',
 'O1',
 'O2',
 'P8',
 'T8',
 'FC6',
 'F4',
 'F8',
 'AF4']

In [3]:
files = os.listdir("201103/")
files

['alpha_wave_03.11.20_23.18.11.json',
 'alpha_wave_03.11.20_23.18.11.md.csv',
 'normal_03.11.20_23.16.54.json',
 'normal_03.11.20_23.16.54.md.csv',
 'readme.md',
 'Sekiguchi_3_03.11.20_23.53.11.json',
 'Sekiguchi_3_03.11.20_23.53.11.md.csv',
 'Sekiguchi_9_03.11.20_23.38.33.json',
 'Sekiguchi_9_03.11.20_23.38.33.md.csv',
 'Sekiguchi_A_03.11.20_23.35.21.json',
 'Sekiguchi_A_03.11.20_23.35.21.md.csv',
 'Sekiguchi_F_03.11.20_23.41.28.json',
 'Sekiguchi_F_03.11.20_23.41.28.md.csv',
 'Sekiguchi_H_03.11.20_23.46.40.json',
 'Sekiguchi_H_03.11.20_23.46.40.md.csv',
 'Sekiguchi_L_03.11.20_23.50.06.json',
 'Sekiguchi_L_03.11.20_23.50.06.md.csv',
 'Sekiguchi_O_03.11.20_23.28.22.json',
 'Sekiguchi_O_03.11.20_23.28.22.md.csv',
 'Sekiguchi_P_03.11.20_23.32.01.json',
 'Sekiguchi_P_03.11.20_23.32.01.md.csv']

In [25]:
O = pd.read_csv("201103/Sekiguchi_3_03.11.20_23.53.11.md.csv", skiprows=1)
O

,Timestamp,EEG.Counter,EEG.Interpolated,EEG.AF3,EEG.F7,EEG.F3,EEG.FC5,EEG.T7,EEG.P7,EEG.O1,...,CQ.T8,CQ.FC6,CQ.F4,CQ.F8,CQ.AF4,CQ.Overall,MOT.CounterMems,MOT.InterpolatedMems,MOT.GyroX,MOT.GyroY
0,1.604415e+09,17.0,0.0,4334.871582,4836.922852,3948.718018,4714.871582,4056.410156,4641.538574,4277.436035,...,4.0,4.0,4.0,4.0,4.0,66.666664,17.0,0.0,1693.0,1677.0
1,1.604415e+09,18.0,0.0,4335.384766,4832.820313,3945.641113,4714.358887,4052.307617,4638.974121,4273.333496,...,4.0,4.0,4.0,4.0,4.0,66.666664,18.0,0.0,1693.0,1676.0
2,1.604415e+09,19.0,0.0,4328.205078,4840.000000,3954.358887,4727.179688,4058.974365,4644.615234,4276.410156,...,4.0,4.0,4.0,4.0,4.0,66.666664,19.0,0.0,1691.0,1678.0
3,1.604415e+09,20.0,0.0,4339.487305,4843.589844,3958.461426,4728.205078,4056.410156,4641.538574,4274.358887,...,4.0,4.0,4.0,4.0,4.0,66.666664,20.0,0.0,1691.0,1678.0
4,1.604415e+09,21.0,0.0,4335.897461,4841.025879,3960.000000,4721.025879,4051.794922,4640.000000,4277.436035,...,4.0,4.0,4.0,4.0,4.0,66.666664,21.0,0.0,1691.0,1678.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18877,1.604415e+09,60.0,0.0,4347.692383,5211.794922,4436.410156,5143.077148,4497.436035,5103.077148,4746.666504,...,4.0,4.0,4.0,4.0,4.0,66.666664,60.0,0.0,1697.0,1669.0
18878,1.604415e+09,61.0,0.0,4352.307617,5215.384766,4443.589844,5148.717773,4524.615234,5108.205078,4747.692383,...,4.0,4.0,4.0,4.0,4.0,66.666664,61.0,0.0,1696.0,1668.0
18879,1.604415e+09,62.0,0.0,4353.333496,5213.846191,4446.153809,5149.743652,4528.205078,5112.820313,4746.153809,...,4.0,4.0,4.0,4.0,4.0,66.666664,62.0,0.0,1696.0,1668.0
18880,1.604415e+09,63.0,0.0,4346.153809,5217.436035,4449.743652,5146.153809,4503.077148,5110.769043,4747.179688,...,4.0,4.0,4.0,4.0,4.0,66.666664,63.0,0.0,1694.0,1668.0


In [26]:
O = O.values[:, 3:22]
O.shape

(18882, 19)

In [27]:
O[:, -1]

array([nan, nan, nan, ..., nan, nan, nan])

In [28]:
np.where(O[:, -1] >= 0), np.where(O[:, -1] >= 0)[0].shape

((array([ 1804,  1850,  1895,  1941,  1987,  2033,  2078,  2124,  2170,
          2216,  2261,  2307,  2353,  2399,  2444,  2490,  2536,  2582,
          2627,  2673,  2719,  2765,  2810,  2856,  2902,  2948,  2993,
          3039,  3085,  3131,  3177,  3222,  3268,  3314,  3360,  3405,
          3451,  3497,  3543,  3588,  3634,  3680,  3726,  3771,  3817,
          3863,  3908,  3954,  4000,  4046,  4091,  4137,  4183,  4229,
          4274,  4320,  4366,  4412,  4457,  4503,  4549,  4595,  4640,
          4686,  4732,  4778,  4823,  4869,  4915,  4960,  5006,  5052,
          5098,  5144,  5189,  5235,  5281,  5327,  5372,  5418,  5464,
          5509,  5555,  5601,  5647,  5693,  5738,  5784,  5830,  5876,
          5921,  5967,  6013,  6058,  6104,  6150,  6196,  6241,  6287,
          6333,  6379,  6424,  6470,  6516,  6562,  6607,  6653,  6699,
          6745,  6790,  6836,  6882,  6928,  6974,  7019,  7065,  7111,
          7156,  7202,  7248,  7294,  7340,  7385,  7431,  7477,

In [30]:
diff = np.diff(np.where(O[:, -1] >= 0)[0])

print("平均", np.round(np.mean(diff) / 128, 4))
print("標準偏差", np.round(np.std(diff) / 128, 4))
print("max", np.round(np.max(diff) / 128, 4))
print("min", np.round(np.min(diff)/ 128, 4))

平均 0.3574
標準偏差 0.0034
max 0.3594
min 0.3516


In [ ]:
plt.figure(figsize=(24, 16))
plt.plot(O[:, 0:-5])
plt.show()

In [ ]:
tt = np.linspace(0, O.shape[0]//128, O.shape[0])
plt.figure(figsize=(24, 16))
plt.plot(tt, O[:, 0:-5])
plt.xlim(30, 30.5)
plt.xlabel("Time[Second]")
plt.ylabel("Amplitude")
plt.show()

In [ ]:
Fs = 128
# firフィルタを適用する
def firFilter(x):
    # fir filter
    b = signal.firwin(511, [1.0/(Fs/2), 15.0/(Fs/2)], pass_zero=False)
    x[:, :-1] = signal.filtfilt(b, 1, x[:, :-1], axis=0)
    return x

In [ ]:
# triggerからNポイント抽出する
N = 128
def triggerExtract(x, trigger):
    triggers = np.where(x[:, -1] == trigger)[0]
    print(len(triggers))
    if trigger == -1:
        triggers = triggers[:-2]
        result = np.zeros((len(triggers), N, x.shape[1]))
    else:
        result = np.zeros((len(triggers), N, x.shape[1]))
    i = 0
    for tmp in triggers:
        tmp = x[tmp:tmp+N].reshape([1, -1, x.shape[1]])
        if tmp.shape[1] == N:
            result[i] = tmp
            i += 1
    return result

In [ ]:
# 波形の加算平均をとる
def WaveAverage(x):
    result = np.zeros((x.shape[1], x.shape[2]))
    for i in range(x.shape[0]):
        result += x[i]
    result = result / x.shape[0]
    return result

In [ ]:
def preprocess(x):
    x = firFilter(x)
    ttrigger = triggerExtract(x, [2, 8])
    ntrigger = triggerExtract(x, [0, 1, 3, 4, 5, 6, 7, 9, 10, 11])
    ttrigger_ave = WaveAverage(ttrigger)
    ntrigger_ave = WaveAverage(ntrigger)
    
    tt = np.linspace(0, 1, ttrigger.shape[1])
    channel = 15
    plt.figure(figsize=(24, 18))
    plt.subplots_adjust(wspace=0.4, hspace=0.8)
    for i in range(len(channels)):
        plt.subplot(8, 4, i+1)
        plt.plot(tt, ttrigger_ave[:, i], 'r', label="target")
        plt.plot(tt, ntrigger_ave[:, i], 'b', label="nontarget")
        plt.xticks(np.arange(0, 1.1, 0.1))
        plt.xlabel("Second[s]")
        plt.ylabel("Amplitude[μV]")
        plt.title(channels[i], fontsize=18)
#     plt.legend()
    return ttrigger_ave, ntrigger_ave

In [ ]:
tt = np.linspace(0, 2, O[:, 0].shape[0])
channel = 15
plt.figure(figsize=(24, 18))
plt.subplots_adjust(wspace=0.4, hspace=0.8)
for i in range(len(channels)):
    plt.subplot(4, 4, i+1)
    plt.plot(tt, O[:, i])
    plt.xlabel("Second[s]")
    plt.ylabel("Amplitude[μV]")
    plt.xticks(np.arange(0, 2.25, 0.25))
    plt.title(channels[i], fontsize=18)

In [ ]:
O = firFilter(O)

In [ ]:
O_target = triggerExtract(O, 2)
O_target = np.concatenate([O_target, triggerExtract(O, 8)], axis=0)

O_nontarget = triggerExtract(O, 0)
for num in [0, 1, 3, 4, 5, 6, 7, 9, 10, 11]:
    O_nontarget = np.concatenate([O_nontarget, triggerExtract(O, num)], axis=0)

O_target.shape, O_nontarget.shape

In [ ]:
tt = np.linspace(0, 1, O_target.shape[1])

plt.figure(figsize=(24, 18))
plt.subplots_adjust(wspace=0.4, hspace=0.4)
for i in range(len(channels)):
    plt.subplot(4, 4, i+1)
    plt.plot(tt, O_target[5, :, i], 'r', label='target')
    plt.plot(tt, O_nontarget[5, :, i], 'b', label='nontarget')
    plt.xticks(np.arange(0, 1.1, 0.1))
    plt.xlabel("Second[s]")
    plt.ylabel("Amplitude[μV]")
    plt.title(channels[i], fontsize=18)
    plt.legend()

In [ ]:
O_target_ave = WaveAverage(O_target)
O_nontarget_ave = WaveAverage(O_nontarget)

O_target_ave.shape, O_nontarget_ave.shape

In [ ]:
tt = np.linspace(0, 1, O_target.shape[1])
channel = 15
plt.figure(figsize=(24, 18))
plt.subplots_adjust(wspace=0.4, hspace=0.4)
for i in range(len(channels)):
    plt.subplot(4, 4, i+1)
    plt.plot(tt, O_target_ave[:, i], 'r', label="target")
    plt.plot(tt, O_nontarget_ave[:, i], 'b', label="nontarget")
    plt.xticks(np.arange(0, 1.1, 0.1))
    plt.xlabel("Second[s]")
    plt.ylabel("Amplitude[μV]")
#     plt.xlim(0.1, 0.6)
    plt.title(channels[i], fontsize=18)
#     plt.legend()

## Target(O) vs non Target(A)

In [ ]:
O_target = triggerExtract(O, 2)
O_target = np.concatenate([O_target, triggerExtract(O, 8)], axis=0)

O_nontarget = triggerExtract(O, 0)
for num in [6]:
    O_nontarget = np.concatenate([O_nontarget, triggerExtract(O, num)], axis=0)

tt = np.linspace(0, 1, O_target.shape[1])

plt.figure(figsize=(24, 18))
plt.subplots_adjust(wspace=0.4, hspace=0.4)
for i in range(len(channels)):
    plt.subplot(4, 4, i+1)
    plt.plot(tt, O_target[5, :, i], 'r', label='target')
    plt.plot(tt, O_nontarget[5, :, i], 'b', label='nontarget')
    plt.xticks(np.arange(0, 1.1, 0.1))
    plt.xlabel("Second[s]")
    plt.ylabel("Amplitude[μV]")
    plt.title(channels[i], fontsize=18)
    plt.legend()

In [ ]:
O_target_ave = WaveAverage(O_target)
O_nontarget_ave = WaveAverage(O_nontarget)

tt = np.linspace(0, 1, O_target.shape[1])
channel = 15
plt.figure(figsize=(24, 18))
plt.subplots_adjust(wspace=0.4, hspace=0.4)
for i in range(len(channels)):
    plt.subplot(4, 4, i+1)
    plt.plot(tt, O_target_ave[:, i], 'r', label="target")
    plt.plot(tt, O_nontarget_ave[:, i], 'b', label="nontarget")
    plt.xticks(np.arange(0, 1.1, 0.1))
    plt.xlabel("Second[s]")
    plt.ylabel("Amplitude[μV]")
    plt.title(channels[i], fontsize=18)
#     plt.legend()

## Target(O) vs non Target(P)

In [ ]:
O_target = triggerExtract(O, 2)
O_target = np.concatenate([O_target, triggerExtract(O, 8)], axis=0)

O_nontarget = triggerExtract(O, 2)
for num in [9]:
    O_nontarget = np.concatenate([O_nontarget, triggerExtract(O, num)], axis=0)

tt = np.linspace(0, 1, O_target.shape[1])

plt.figure(figsize=(24, 18))
plt.subplots_adjust(wspace=0.4, hspace=0.4)
for i in range(len(channels)):
    plt.subplot(4, 4, i+1)
    plt.plot(tt, O_target[5, :, i], 'r', label='target')
    plt.plot(tt, O_nontarget[-5, :, i], 'b', label='nontarget')
    plt.xticks(np.arange(0, 1.1, 0.1))
    plt.xlabel("Second[s]")
    plt.ylabel("Amplitude[μV]")
    plt.title(channels[i], fontsize=18)
    plt.legend()

In [ ]:
O_target_ave = WaveAverage(O_target)
O_nontarget_ave = WaveAverage(O_nontarget)

tt = np.linspace(0, 1, O_target.shape[1])
channel = 15
plt.figure(figsize=(24, 18))
plt.subplots_adjust(wspace=0.4, hspace=0.4)
for i in range(len(channels)):
    plt.subplot(4, 4, i+1)
    plt.plot(tt, O_target_ave[:, i], 'r', label="target")
    plt.plot(tt, O_nontarget_ave[:, i], 'b', label="nontarget")
    plt.xticks(np.arange(0, 1.1, 0.1))
    plt.xlabel("Second[s]")
    plt.ylabel("Amplitude[μV]")
    plt.title(channels[i], fontsize=18)
#     plt.legend()